In [1]:
import os 
os.chdir("../")

### Load the data

In [129]:
import pandas as pd
df = pd.read_csv("data/dataset.csv")

C:\Users\windows 10\AppData\Local\Temp\ipykernel_9928\685072172.py:2: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/dataset.csv")


### Take 10% of the data

In [130]:
df = df.sample(frac=0.1, random_state=42)  

In [114]:
import pandas as pd

# Load the downloaded postal code data
postal_data = pd.read_csv("data/ZA.txt", sep='\t', header=None, names=['CountryCode', 'PostalCode', 'PlaceName', 'Admin1Name', 'Admin1Code', 'Admin2Name', 'Admin2Code', 'Admin3Name', 'Admin3Code', 'Latitude', 'Longitude', 'Accuracy'])

# Create a mapping of postal codes to city names
postal_code_to_city = dict(zip(postal_data['PostalCode'], postal_data['PlaceName']))
# Add a new column in your dataset by mapping the postal code to city names
df['CityName'] = df['PostalCode'].map(postal_code_to_city)


### Selecting important columns

In [115]:
from scripts.data_preparation import feature_engineering


df = feature_engineering(df)

In [116]:

selected_columns = ['CityName','RiskFactor','VehicleAge', 'MainCrestaZone', 'SubCrestaZone', 'mmcode', 'Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType','TotalPremium', 'TotalClaims']

df = df[selected_columns]

### Data Preparation: 

In [ ]:
from scripts.preprocessing import  replace_missing_with_mean, replace_missing_with_mode
 


numerical_cols = df.select_dtypes(include=['int64', 'float64'])
categorical_cols = df.select_dtypes(include='object')

df = replace_missing_with_mode(df,categorical_cols)
df = replace_missing_with_mean(df,numerical_cols.columns)



In [127]:
import pandas as pd
from scipy import stats

def remove_outliers(df, threshold=0.01):
    """
    Remove outliers from the DataFrame based on Z-scores.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    threshold (float): The Z-score threshold to identify outliers.
    
    Returns:
    pd.DataFrame: DataFrame with outliers removed.
    """
    # Calculate Z-scores
    z_scores = stats.zscore(df.select_dtypes(include=['float64', 'int64']))
    
    # Create a boolean mask for outliers
    outliers_mask = (abs(z_scores) > threshold).all(axis=1)
    
    # Filter out the outliers
    df_cleaned = df[~outliers_mask]
    
    return df_cleaned



# Remove outliers
df = remove_outliers(df)




In [31]:
from sklearn.preprocessing import MinMaxScaler


# Columns to scale
columns_to_scale = [
     'RiskFactor', 'VehicleAge', 'mmcode', 'Cylinders', 
    'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CustomValueEstimate', 
    'CapitalOutstanding', 'SumInsured', 
    'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims'
]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])


#### Encoding Categorical Data:
* Convert categorical data into a numeric format using one-hot encoding or label encoding to make it suitable for modeling.

In [32]:
from scripts.data_preparation import encode_categorical_data


df = encode_categorical_data(df)


#### Train-Test Split:
* Divide the data into a training set (for building the model) and a test set (for validating the model) using a 70:30 


In [33]:
from scripts.data_preparation import train_test_splitting


target_cols = ['TotalPremium', 'TotalClaims']
X_train, X_test, y_train, y_test = train_test_splitting(df, target_cols)

### Model Building
* Implement Linear Regression, Random Forests, and XGBoost models


In [ ]:

from scripts.models import train_and_evaluate_models


results = train_and_evaluate_models(X_train, X_test, y_train['TotalPremium'], y_test['TotalPremium'])
    
# Print results for each model
for model_name, metrics in results.items():
    print(f"{model_name}: MSE = {metrics['MSE']}, R2 = {metrics['R2']}")